#Team Armadillo — Governance Trace Dashboard
This notebook visualizes logged governance data produced by the Enterprise-Ready AI pipeline.
Each log represents a model inference with an associated risk score, captured for audit and evaluation.

Run this inside **IBM watsonx.ai** or locally after generating several governance logs.

In [ ]:
# ================================================================
# SECTION 1 — IMPORTS AND SETUP
# ================================================================
import os, json, pandas as pd, matplotlib.pyplot as plt

log_folder = "../data"
logs = [f for f in os.listdir(log_folder) if f.startswith("governance_") and f.endswith(".json")]
print(f"Found {len(logs)} governance log(s)")

In [ ]:
# ================================================================
# SECTION 2 — LOAD LOGS INTO A DATAFRAME
# ================================================================
records = []
for file in sorted(logs):
    path = os.path.join(log_folder, file)
    with open(path) as f:
        data = json.load(f)
        record = {
            "timestamp": int(file.split("_")[1].split(".")[0]),
            "risk_score": data.get("risk_score", None),
            "summary": data.get("summary", "")[:120] + "..."
        }
        records.append(record)

if not records:
    print("No governance logs found — run the Enterprise AI notebook first.")
else:
    df = pd.DataFrame(records).sort_values("timestamp")
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
    display(df.head())

In [ ]:
# ================================================================
# SECTION 3 — VISUALIZE RISK TRENDS
# ================================================================
if records:
    plt.figure(figsize=(8,4))
    plt.plot(df['datetime'], df['risk_score'], marker='o', linewidth=2)
    plt.title('Risk Score Trend Across Inferences', fontsize=14)
    plt.xlabel('Timestamp')
    plt.ylabel('Risk Score (0–1)')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("Skipping visualization — no data available.")

In [ ]:
# ================================================================
# SECTION 4 — BASIC STATS SUMMARY
# ================================================================
if records:
    avg = round(df['risk_score'].mean(), 3)
    high = df['risk_score'].max()
    low = df['risk_score'].min()
    print(f"Average Risk Score: {avg}")
    print(f"Highest: {high}  🔻 Lowest: {low}")
else:
    print("No records to summarize.")